## Introduction to SQLite Practice Challenge

In [ ]:
import hashlib
import shutil
import sqlite3
from unittest import TestCase, TestLoader, TextTestRunner

import requests

def runTest(case):
    suite = TestLoader().loadTestsFromModule(case())
    TextTestRunner().run(suite) 

## This Week's Aims
Despite the "[NoSQL movement](https://pragprog.com/book/rwdata/seven-databases-in-seven-weeks)", SQL (structured query language) for databases remains ubiquitous. Python's standard library speaks a dialect of it called SQLite, that's also used by Android apps. We'll use it to explore some data gathered during the development of the "MobileMiner" app for the "[Our Data, Ourselves](https://big-social-data.net/)" project. We'll look at the network behaviour of some Android apps, and the trail of mobile cell towers we leave behind. Later on, you can read about it [here](https://pdfs.semanticscholar.org/873b/2c14c22e232911166f215971bc2acaf8f00b.pdf) and [here](http://kdd.isti.cnr.it/pap2017/papers/PAP_2017_paper_3.pdf).

Here are some utility functions to grab the databases we need and show what they contain.

In [ ]:
def getSqliteFiles():
    files = ['https://github.com/kingsBSD/sqlite-example/raw/master/MobileMinerData.sqlite',
        'https://github.com/kingsBSD/sqlite-example/raw/master/celldata.sqlite']
    filenames = [f.split('/')[-1] for f in files]
    for url, name in zip(files,filenames):
        response = requests.get(url)
        with open(name, 'wb') as f:
            f.write(response.content)
    return filenames

def copySqliteFiles(files):
    dest = ['.'.join([f.split('.')[0]+'_copy','sqlite']) for f in files]
    for orig,copy in zip(files,dest):
        shutil.copyfile(orig,copy)
    return dest

def getDbTables(cursor):
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return [t[0] for t in cursor.fetchall()]

def getTableFields(cursor, table):
    try:
        cursor.execute('SELECT * FROM '+table)
    except:
        return None
    return [f[0] for f in cursor.description] 


You only need to run the next cell once. It might take a little while to get all the data.

In [ ]:
sqlite_original_files = getSqliteFiles()

The next cell makes working copies of the original database files. Run it again if you mess them up.

In [ ]:
sqlite_files = copySqliteFiles(sqlite_original_files)
sqlite_files

We create connections and cursors for the two databases, just like in standard module [documentation](https://docs.python.org/3/library/sqlite3.html).

In [ ]:
mobile_miner_conn = sqlite3.connect('MobileMinerData_copy.sqlite')
mobile_miner_cursor = mobile_miner_conn.cursor()
cell_tower_conn = sqlite3.connect('celldata_copy.sqlite')
cell_tower_cursor = cell_tower_conn.cursor()

The first database was downloaded from an Android 'phone used to develop the MobileMiner app. Let's get a list of the tables:

In [ ]:
getDbTables(mobile_miner_cursor)

The "socket" table is quite interesting, it logs when various apps opened network sockets. The name of the app is stored in the "process" column. Also of interest is the time when the socket was opened, the network protocol used and the IP port. (You are probably looking at this notebook via TCP port 8888.)

In [ ]:
getTableFields(mobile_miner_cursor, 'socket')

### Part 1: Basic Selecting
The most basic SQL statement is "SELECT". We pass the SQL code with our cursor's "execute" method. This should return all the columns from all the rows from the "gsmlocation" table. We get the data returned with the "fetchall" method.

In [ ]:
mobile_miner_cursor.execute("SELECT * FROM gsmlocation")
mobile_miner_cursor.fetchall()

Oh dear, that table is empty! We might do something about that later. This time we'll select only the "process" column from the "socket" table. There will be loads of network events for Chrome. If we specify the "DISTINCT" modifier, we make sure all the returned rows are unique. In short, this returns all the apps in the socket table.

In [ ]:
mobile_miner_cursor.execute('SELECT DISTINCT process FROM socket')
mobile_miner_cursor.fetchall()

Here, we return the process (the app) and the network protocol. We use "ORDER BY" to sort first by the protocol,
then by the process name, so all the apps that used TCP are listed first. We get a list of tuples, one for each row returned. (I'd tell you a UDP [joke](https://yakking.branchable.com/posts/udp/), but you might not get it.)

In [ ]:
mobile_miner_cursor.execute('SELECT DISTINCT process, protocol FROM socket ORDER BY protocol, process')
mobile_miner_cursor.fetchall()

Now look at the "networktraffic" table. It was generated using standard Android API calls, unlike the cheap hack used to produce the "socket" table. It's more reliable, but it doesn't tell us as much. There's no information about the ports or protocols.

In [ ]:
getTableFields(curs,'networktraffic')

Here's where you come in. Complete the "getTrafficProcesses" function, which takes a SQLite cursor as it's only argument. It should return a unique list of all the processes in the "networktraffic" table, without worrying about sorting. A single "SELECT" statement is needed. You should return a list, not a list of tuples. The cell will test the function for you.

In [ ]:
class TestGetTrafficProcesses(TestCase):
    """Don't worry too much about how these tests work...
    (...unless you *really* want to.)
    """
    
    def testLength(self):
        self.assertEqual(len(getTrafficProcesses(mobile_miner_cursor)),28)

    def testContent(self):
        # This is a *viciously* over-engineered way to prevent you from cheating.
        procs = getTrafficProcesses(mobile_miner_cursor)
        indices  = [4,8,17,24]
        hashed_procs = [hashlib.sha224(bytes(procs[i],'ascii')).hexdigest() for i in indices]
        self.assertEqual(hashed_procs,['2fb69a97e6ae8c23549dae4eb1699ca08b4b3c6e8427deffc647a695',
            '0a27edaf745f7efd409cf5f2dc4f02c69bdcd048842a044d9df4394b',
            '67987888fac2a46894a8014ed71841bcd4c7696656da087bbb2ff98f',
            '0873cf983d06bd5526db5c9b98e91fa6a9ed6aec75944ef543dfc05d'])
        
def getTrafficProcesses(cursor):
    # Your code here:
    pass
    
runTest(TestGetTrafficProcesses)    

See what your function returns. There are loads more apps in the "networktraffic" table than in the socket table.

In [ ]:
getTrafficProcesses(mobile_miner_cursor)

Now you're going to need a very slightly more complex query, then manipulate the results with a little Python. The function "getProcessPorts" should return a dictionary whose keys are all the processes in the "socket" table. The values should be lists of the ports used by each process. The order of the ports in each list doesn't matter.

In [ ]:
class TestGetProcessPorts(TestCase):
    """Don't worry too much about how these tests work...
    (...unless you *really* want to.)
    """
    
    def testLength(self):
        self.assertEqual(len(getProcessPorts(mobile_miner_cursor)),13)
        
    def testContent(self):
        # This is another *viciously* over-engineered way to prevent you from cheating.
        results = getProcessPorts(mobile_miner_cursor)
        hashCheck = lambda p: hashlib.sha224(bytes('_'.join(sorted(results[p])),'ascii')).hexdigest()
        self.assertEqual(hashCheck('com.skype.raider'),'9ba1885f4e8b5b5724b4f20fb2143d8e41be8facba8d155cbc313509')
        self.assertEqual(hashCheck('com.facebook.katana'),'6c905a484091b8fd5e27b0cbdb51751f6de1f15f2f0d9d1b06149e92')

def getProcessPorts(cursor):
    # Your code here...
    pass
    
runTest(TestGetProcessPorts)

## Part 2: WHERE Clauses
We can filter the results of queries with a "WHERE" clause. This one returns all the apps that ever opened a network socket on a Thurdsay. Notice that the string 'Thu' has to be in quotes.

In [ ]:
mobile_miner_cursor.execute("SELECT DISTINCT process FROM socket WHERE day = 'Thu'")
mobile_miner_cursor.fetchall()

We can have multiple conditions in a "WHERE" clause using boolean operators. ("AND", "OR", etc...) Here are all the apps that ever opened a network socket at the  weekend are returned. Both the process and the day of the week are selected.

In [ ]:
mobile_miner_cursor.execute("SELECT DISTINCT process,day FROM socket WHERE day = 'Sat' OR day = 'Sun'")
mobile_miner_cursor.fetchall()

If you add "DESC" to "ORDER BY", the results are returned in descending order.

In [ ]:
mobile_miner_cursor.execute('SELECT DISTINCT process FROM socket ORDER BY process DESC')
mobile_miner_cursor.fetchall()

The standard module [documentation](https://docs.python.org/3/library/sqlite3.html) tells you how to inject values into SQL queries. Use a "?" as a placeholder for each value, and pass them in order using a tuple. Here's all Twitter did on port 53:

In [ ]:
values = ('com.twitter.android',53)
mobile_miner_cursor.execute("SELECT * FROM socket WHERE process=? AND port=?", values)
mobile_miner_cursor.fetchall()

In [ ]:
#If your tuple only has one item, it needs a trailing comma:
only_one_answer = (42,)

Now you can write a "WHERE" clause of your own. "getUdpEvents" takes a cursor and an Android package. It should
return a list of timestamps of when UDP sockets were opened for that package in ascending order.

In [ ]:
class TestGetUdpEvents(TestCase):
    """Don't worry too much about how these tests work...
    (...unless you *really* want to.)
    """
        
    def testLength(self):
        self.assertEqual(len(getUdpEvents(mobile_miner_cursor,'com.skype.raider')),7)
        
    def testContent(self):
        hashCheck = lambda r: hashlib.sha224(bytes(' '.join(r),'ascii')).hexdigest()
        result = getUdpEvents(mobile_miner_cursor,'com.skype.raider')
        self.assertEqual(hashCheck(result), '85f1bca843c87aa91bc0650893db8275ba4251c545c7da2ff500898f')

def getUdpEvents(cursor, package):
    # Your code here:
    pass
    
runTest(TestGetUdpEvents)

## Part 3: Aggregation Functions
SQL can apply functions to the columns it returns. Here we'll count all the times Chrome opened a socket on port 80, using insecure HTTP:

In [ ]:
mobile_miner_cursor.execute("SELECT COUNT(port) FROM socket WHERE port=80 AND process='com.android.chrome'")
mobile_miner_cursor.fetchall()

We'll do it again for sockets using secure HTTPS on port 443. We should be able to return both rows with one query, which we'll fix very soon.

In [ ]:
mobile_miner_cursor.execute("SELECT COUNT(port) FROM socket WHERE port=443 AND process='com.android.chrome'")
mobile_miner_cursor.fetchall()

There are lots of other functions. Use "MAX" to return the largest/latest timestamp for a network socket for the given process. Your function should return a single timestamp as a string.

In [ ]:
class TestMostRecentSocket(TestCase):
    
    def testResult(self):
        apps=['com.google.android.gsf.login','com.google.android.gms','com.google.android.gm']
        hsh = hashlib.sha224(bytes(' '.join(map(lambda a: mostRecentSocket(mobile_miner_cursor, a),apps)),
            'ascii')).hexdigest()
        self.assertEqual(hsh,'7f828e6599d22aaf0dba75e8081727c2c86094abf8fb99367ffb0bfe')

def mostRecentSocket(cursor, process):
    # Your code here:
    pass

runTest(TestMostRecentSocket)

## Part 4: GROUP BY
Now we can the total number of sockets for each port used by Chrome. We select the port and the count of the port.
Using "GROUP BY port" means that the count is repeated for each port number, we don't just count all the sockets at once. We can see that more sockets were opened using insecure HTTP on port 80 than secure HTTPS on port 443:

In [ ]:
mobile_miner_cursor.execute("""SELECT port, COUNT(port) FROM socket
    WHERE process='com.android.chrome' GROUP BY port""")
mobile_miner_cursor.fetchall()

Can we do that for all the apps? Will the next query work?

In [ ]:
mobile_miner_cursor.execute("SELECT process, port, COUNT(port) FROM socket GROUP BY port")
mobile_miner_cursor.fetchall()

That just looks *wrong*. Nearly all the apps are missing, and Chrome looks like it's too many sockets on port 443. We need to group by process *and* port:

In [ ]:
mobile_miner_cursor.execute("SELECT process, port, COUNT(port) FROM socket GROUP BY process, port")
mobile_miner_cursor.fetchall()

That's much better, all the apps are back and the numbers for Chrome look right. We can use "AS" to alias "COUNT(port)" as "count". Now we can sort the results by the app names and have the most active ports listed first.

In [ ]:
mobile_miner_cursor.execute(
    "SELECT process, port, COUNT(port) AS count FROM socket GROUP BY process, port ORDER BY process, count DESC")
mobile_miner_cursor.fetchall()

Now you can have a go at using "GROUP BY". Use the "networktraffic" table. Return a list of dictionaries whose keys are "app" and "count". "app" is the process, "count" is the number of network-traffic events. The most active apps should be returned first. The previous query should give you a clue.

In [ ]:
class TestBusiestAppTraffic(TestCase):
    
    def testAppOrder(self):
        hsh = hashlib.sha224(bytes(' '.join([a['app'] for a in busiestAppTraffic(mobile_miner_cursor)]),
            'ascii')).hexdigest()
        self.assertEqual(hsh, 'ef533305f05261e0867782d66de6066a7451602e0050535b38ba8aad')
    
    def testCounts(self):
        hsh = hashlib.sha224(bytes(' '.join([a['count'].__str__()
            for a in busiestAppTraffic(mobile_miner_cursor)]),'ascii')).hexdigest()
        self.assertEqual(hsh, '779e97159f1c65fcc66f6e2232e4f24c4efa88cf6e42c7f7da1506ed')

def busiestAppTraffic(cursor):
    # Your code here:
    pass

runTest(TestBusiestAppTraffic)

That one was hard work, let's look at the results:

In [ ]:
busiestAppTraffic(mobile_miner_cursor)

## Part 5: Investigating Mobile Cell Towers
Take a look at the "gsmcell" table. The key columns are the mobile country code, the mobile network code, the local area code, and the cell ID.

In [ ]:
getTableFields(mobile_miner_cursor, 'gsmcell')

"miner_cells" is a list of all the cell towers the Android 'phone connected to: 

In [ ]:
mobile_miner_cursor.execute("SELECT DISTINCT mcc,mnc,lac,cid FROM gsmcell WHERE NOT cid = 'None'")
miner_cells = mobile_miner_cursor.fetchall()
miner_cells[0:5]

Writing "WHERE" clauses with four conditions is tedious, so here's the "whereAll" function to help, if you like. Have a look at what it does, don't worry *too* much if you can't see how it does it. (Are you addicted to [zip](https://docs.python.org/3.6/library/functions.html#zip) yet?)

In [ ]:
def whereAll(values, columns=['mcc','mnc','lac','cid']):
    return " WHERE " + ' AND '.join([' = '.join([col,val]) for val,col in zip(values,columns)])

whereAll(miner_cells[0])

In [ ]:
getTableFields(cell_tower_cursor, 'gsmlocation')

We're going to use the cell_tower_cursor now, the database has all the UK cell towers known to the [OpenCellID database](https://opencellid.org/) Look up the lattitudes and longitudes of every cell in "miner_cells". Return a list of (lat,long) tuples. Not every cell will be known to the datbase, do not return any empty rows.

In [ ]:
class TestGetCellLocation(TestCase):
    
    def testCount(self):
        self.assertEqual(len(getCellLocations()),13)
        
    def testValues(self):
        lat, lon = list(zip(*getCellLocations()))
        lon_hash = hashlib.sha224(bytes(' '.join(lon),'ascii')).hexdigest()
        self.assertEqual(lon_hash, '9d90b67b1a4a3cd2d9772bff7158c8cc8002c62010a25fcf26b2826e')
        lat_hash = hashlib.sha224(bytes(' '.join(lat),'ascii')).hexdigest()
        self.assertEqual(lat_hash, 'ffbc715902d50191bacde877603f3cb2e463081c02e775ec59666b66')
        
def getCellLocations(cursor=cell_tower_cursor, towers=miner_cells):
    # Your code here...
    pass

runTest(TestGetCellLocation)

The function takes a little while to run, so we'll keep the results in "miner_locations".

In [ ]:
miner_locations = getCellLocations()

Now we've got the cell locations, let's plot them on a map. Make sure you have the [Folium library](https://folium.readthedocs.io/en/latest/) installed.

In [ ]:
import statistics

import folium

lattitude, longitude = list(zip(*miner_locations))
# The locations are returned as strings. Turn them back into floats:
lat = list(map(float,lattitude))
lon = list(map(float,longitude))
centre = [statistics.mean(lat),statistics.mean(lon)]
cell_map = folium.Map(location=centre, zoom_start=12)
for point in zip(lat,lon):
    folium.Marker(point).add_to(cell_map)
cell_map

## Part 6: Inserting Data
Let's put the cell location data into the empty "gsmlocation" table where it can be of some use. You'll need the
columns 'mcc', 'mnc', 'lac', 'cid', 'lat', 'long'. Combine the "miner_cells" and "miner_locations" lists to form the tuples. The test will drop and recreate the table for you each time you run it. The standard library documentation will be a big help. Remember to use "mobile_miner_conn.commit()".

In [ ]:
# Clue: Merging tuples is easy!
(1,2,3) + (4,5,6)

In [ ]:
class TestPushCellLOcations(TestCase):
    
    def setUp(self):

            try:
                mobile_miner_cursor.execute('DROP TABLE gsmlocation')
                mobile_miner_conn.commit()
            except:
                pass
            
            try:
                mobile_miner_cursor.execute('''CREATE TABLE gsmlocation(id INTEGER PRIMARY KEY,
                mcc TEXT, mnc TEXT, lac TEXT, cid TEXT, lat TEXT, long TEXT)''')
                mobile_miner_conn.commit()
            except:
                pass
            
    def testContent(self):
        pushCellLOcations()
        mobile_miner_cursor.execute("SELECT * FROM gsmlocation")
        content = ' '.join([' '.join(r[1:]) for r in mobile_miner_cursor.fetchall()])
        content_hash = hashlib.sha224(bytes(content,'ascii')).hexdigest()
        self.assertEqual(content_hash,'500c72d0f6f5993965d514cc42c88ab2a2693249c8de9763c551269f')

def pushCellLOcations(cursor=mobile_miner_cursor, cells=miner_cells, locations=miner_locations):
    # Your code here:
    pass
    
runTest(TestPushCellLOcations)

Now we can join the "gsmcell" and "gsmlocation" location tables togther, into one big table. The rows are combined so that all the cell IDs match. This gives the time and location of each visit to a known cell tower:

In [ ]:
mobile_miner_cursor.execute('SELECT time, lat, long FROM gsmcell NATURAL JOIN gsmlocation')
mobile_miner_cursor.fetchall()[0:10]

In [ ]:
def testAll():
    all_the_tests = [TestGetTrafficProcesses, TestGetProcessPorts, TestGetUdpEvents, TestBusiestAppTraffic,
        TestGetCellLocation, TestPushCellLOcations]
    total_tests = 0
    total_passes = 0
    runner = TextTestRunner(verbosity=0)
    for test in all_the_tests:
        suite = TestLoader().loadTestsFromModule(test())
        count = suite.countTestCases()
        total_tests += count
        result = runner.run(suite)
        total_passes += count - len(result.errors) - len(result.failures)
        print()
    return total_tests, total_passes

tests, passes = testAll()

In [ ]:
print('tests run:', tests)
print('tests passed:', passes)

![xkcd 327](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)
([xkcd](https://xkcd.com/327/))